In [1]:
%%sh
python convert.py

processing /home/jovyan/jsons/20_03_21__04_00_04.json
56112 records read from json
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|   shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id| type|              datetm|                 url|
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|karusel| Антиперспирант-к...|    49|199.99|     null|     4081.43|            null|    null| 15|   1007746|piece|2020-03-21 05:48:...|/products/35828-a...|
|  metro| Антиперспирант-к...|    49| 111.9|    190.0|     2283.67|        3877.551|   41.11| 15|    137598|piece|2020-03-21 04:14:...|/products/35828-a...|
|karusel| Бальзам для особ...|   225| 97.99|     null|      435.51|            null|    null| 15|   1019125|piece|2020-03-21 05:48:.

20/03/21 12:38:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [4]:
from pyspark.sql.dataframe import DataFrame
DataFrame.old_show = getattr(DataFrame, 'old_show', DataFrame.show)
def show(self, n=20, truncate=20):
    self.persist()
    print(self.count())
    self.old_show(n, truncate=truncate)
    return self

DataFrame.show = show

import random
def sql(self, q):
    name = 'table_{}'.format(random.randint(0000, 9999))
    self.createOrReplaceTempView(name)
    q = q.format(T=name)
    print(q)
    return df.sql_ctx.sql(q)
DataFrame.sql = sql


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()

df = spark.read.parquet('store').withColumn('date', F.date_format(F.date_trunc('DAY', F.col('datetm')), 'YY-MM-dd')).show()



275668
+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+------+--------------------+--------------------+--------+
| shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id|  type|              datetm|                 url|    date|
+-----+--------------------+------+------+---------+------------+----------------+--------+---+----------+------+--------------------+--------------------+--------+
|metro|Проростки Микс Кр...|    70|169.89|     null|      2427.0|            null|    null|  1|    123000| piece|2020-03-16 19:36:...|/products/30784-p...|20-03-16|
|metro|Чай черный байхов...|   100| 799.0|     null|      7990.0|            null|    null| 12|    123492| piece|2020-03-16 19:42:...|/products/16059-c...|20-03-16|
|metro|Оливки с перцем T...|   300|  85.0|     null|      283.33|            null|    null|  7|    124369| piece|2020-03-16 20:09:...|/products/17355-o...|20-03-16|
|me

In [6]:
# total products per shop
df.groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|    8991|    8689|    8629|    8660|    8680|
|  metro|   19599|   19645|   19365|   19702|   19727|
|karusel|   12501|   13139|   12450|   12642|   12803|
|  lenta|   14394|   14176|   13748|   14087|   14041|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [7]:
# total products with discounts per shop 
df.where('old_price is not null').groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|     997|     934|     908|     783|     759|
|  metro|    2707|    2813|    2762|    2833|    2988|
|karusel|    1836|    2022|    1629|    1141|    1802|
|  lenta|    1513|    1531|    1490|    1430|    1305|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [8]:
# total average price per shop 
df.groupBy("shop").pivot("date").agg(F.sum('price')/F.count('product_id')).show()


4
+-------+------------------+------------------+------------------+------------------+------------------+
|   shop|          20-03-16|          20-03-17|          20-03-18|          20-03-19|          20-03-21|
+-------+------------------+------------------+------------------+------------------+------------------+
| prisma|188.10001347850155|188.34344018157324|189.03996892411402|192.07068617674275|192.57394608264696|
|  metro|  291.260432704589| 287.4751096981541| 289.6296090938372| 289.3170404041156|284.33816950983623|
|karusel|163.46044942153262|162.95179831070737|166.14586250631686| 168.7916300962232|166.81134088776088|
|  lenta|212.23314121221028|210.92874393766877|213.27697006009296|213.59038444901407| 213.3129290605269|
+-------+------------------+------------------+------------------+------------------+------------------+



DataFrame[shop: string, 20-03-16: double, 20-03-17: double, 20-03-18: double, 20-03-19: double, 20-03-21: double]